In [ ]:
!pip install split-folders

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras
import matplotlib.pyplot as plt
import os
from sklearn.metrics import confusion_matrix , ConfusionMatrixDisplay, classification_report
from tensorflow.keras.models import model_from_json

import splitfolders

In [ ]:
splitfolders.ratio("../input/model-3-interesp-sin-con-l", output="./data", seed=1000, ratio=(.85, .15))

In [ ]:
TRAIN_DIR = './data/train'
TEST_DIR = './data/val'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

color_mode="rgb"  #rgb       grayscale
channels = 3
width = 1706
height = 1165
batch_size = 25

train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2)

test_datagen = ImageDataGenerator(
        rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    color_mode=color_mode,
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR, # same directory as training data
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    color_mode=color_mode,
    subset='validation') # set as validation data


test_generator = test_datagen.flow_from_directory(TEST_DIR, 
                                                   target_size=(height, width), 
                                                   batch_size = 1, 
                                                   shuffle=False,
                                                   color_mode=color_mode,
                                                   class_mode="binary")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalMaxPooling2D, BatchNormalization, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LeakyReLU
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.regularizers import l2

model = Sequential()
model.add(Conv2D(16, (5, 5), input_shape=(width, height, channels), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(GlobalMaxPooling2D())

model.add(Dense(1024, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))


model.summary()

epochs = 500



model.compile(loss=tensorflow.keras.losses.binary_crossentropy,
              optimizer=tensorflow.keras.optimizers.Adam(learning_rate=0.00005), #learning_rate=0.00005
              metrics=['accuracy'])

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
    
    
filepath="weights-improvement-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
callbacks_list = [checkpoint]

In [ ]:
h = model.fit(train_generator, epochs=epochs, validation_data=validation_generator, callbacks=callbacks_list)

In [ ]:
history=h
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
def evaluate_set(set_generator):
    model.evaluate(set_generator)
    
    pred =  model.predict(set_generator)
    y_pred = [1 * (x[0]>=0.5) for x in pred]

    conf = confusion_matrix(set_generator.classes, y_pred)

    disp = ConfusionMatrixDisplay(confusion_matrix=conf, display_labels=np.unique(set_generator.labels))

    disp = disp.plot(cmap=plt.cm.Blues, xticks_rotation='0')
    plt.show()
    
    print(classification_report(set_generator.classes, y_pred))
    
    TP = conf[1][1]
    TN = conf[0][0]
    FP = conf[0][1]
    FN = conf[1][0]
    conf_accuracy = (float (TP+TN) / float(TP + TN + FP + FN))
    print('accuracy:', round(conf_accuracy,4))
    conf_specificity = (TN / float(TN + FP))
    print('specificity:', round(conf_specificity,4))
    conf_sensitivity = (TP / float(TP + FN))
    print('sensitivity:', round(conf_sensitivity,4))
    conf_precision = (TP / float(TP + FP))
    print('precision:', round(conf_precision,4))

In [ ]:
evaluate_set(train_generator) 

In [ ]:
evaluate_set(validation_generator) 

In [ ]:
evaluate_set(test_generator)